<a href="https://colab.research.google.com/github/anfalaugust/CSC496_Models/blob/main/CSC496_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.listdir("/content/drive/MyDrive/")

!unzip "/content/drive/MyDrive/CSC496/Diseases of date palm leaves dataset.zip" -d "/content/dataset"

import tensorflow as tf
os.listdir("/content/dataset")

dataset_path = "/content/dataset/Diseases of date palm leaves dataset/Infected Date Palm Leaves Dataset/Processed"

train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    image_size=(224, 224),
    batch_size=32
)

print("Classes:", train_ds.class_names)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  /content/drive/MyDrive/CSC496/Diseases of date palm leaves dataset.zip
replace /content/dataset/Diseases of date palm leaves dataset/Infected Date Palm Leaves Dataset/Processed/1. Potassium Deficiency/M9 (101).png? [y]es, [n]o, [A]ll, [N]one, [r]ename: Found 3089 files belonging to 9 classes.
Classes: ['1. Potassium Deficiency', '2. Manganese Deficiency', '3. Magnesium Deficiency', '4. Black Scorch', '5. Leaf Spots', '6. Fusarium Wilt', '7. Rachis Blight', '8. Parlatoria Blanchardi', '9. Healthy sample']


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [ ]:
# ==============================
# ResNet50: Load Data -> Train -> Evaluate (One Cell)
# ==============================


# 1) Paths & Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42
EPOCHS = 10

# 2) Load dataset + Train/Validation split (80/20)
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)

# 3) Optimize pipeline
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# 4) Build Model (Transfer Learning)
base_model = tf.keras.applications.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)
base_model.trainable = False

model = keras.Sequential([
    layers.Rescaling(1./255),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# 5) Compile
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 6) Train
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

# 7) Evaluate
val_loss, val_acc = model.evaluate(val_ds)
print("✅ Validation Accuracy:", val_acc)
print("✅ Validation Loss:", val_loss)

Found 3089 files belonging to 9 classes.
Using 2472 files for training.
Found 3089 files belonging to 9 classes.
Using 617 files for validation.
Epoch 1/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 34s 249ms/step - accuracy: 0.2179 - loss: 2.2372 - val_accuracy: 0.2480 - val_loss: 1.9375
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 9s 110ms/step - accuracy: 0.2658 - loss: 1.9798 - val_accuracy: 0.2496 - val_loss: 1.8968
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 8s 107ms/step - accuracy: 0.2883 - loss: 1.9093 - val_accuracy: 0.3209 - val_loss: 1.8567
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 8s 106ms/step - accuracy: 0.2927 - loss: 1.8789 - val_accuracy: 0.3193 - val_loss: 1.8499
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 8s 105ms/step - accuracy: 0.3065 - loss: 1.8785 - val_accuracy: 0.2966 - val_loss: 1.8357
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 8s 104ms/step - accuracy: 0.3040 - loss: 1.8803 - val_accuracy: 0.3015 - val_loss: 1.8113
Epoch 7/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 8s 106ms/step - accuracy: 0.3070 - loss: 1.8438 

In [ ]:
# ==============================
# MobileNetV2: Load Data -> Train -> Evaluate (One Cell)
# ==============================

# 1) Paths & Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42
EPOCHS = 10

# 2) Load dataset + Train/Validation split (80/20)
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)

# 3) Optimize pipeline
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# 4) Build Model (Transfer Learning)
base_model = tf.keras.applications.MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)
base_model.trainable = False

model = keras.Sequential([
    layers.Rescaling(1./255),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# 5) Compile
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 6) Train
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

# 7) Evaluate
val_loss, val_acc = model.evaluate(val_ds)
print("✅ Validation Accuracy:", val_acc)
print("✅ Validation Loss:", val_loss)

Found 3089 files belonging to 9 classes.
Using 2472 files for training.
Found 3089 files belonging to 9 classes.
Using 617 files for validation.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 54s 403ms/step - accuracy: 0.3408 - loss: 1.9836 - val_accuracy: 0.6872 - val_loss: 0.9837
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.6272 - loss: 1.1056 - val_accuracy: 0.7407 - val_loss: 0.7585
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.7042 - loss: 0.8792 - val_accuracy: 0.8055 - val_loss: 0.6382
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.7871 - loss: 0.6434 - val_accuracy: 0.8331 - val_loss: 0.5348
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.8172 - loss: 0.5581 - val_accuracy: 0.8363 - val_loss: 0.5197
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - accuracy: 0.8411 - loss: 0.5166 - val_accuracy: 0.8703 - val_loss: 0.4446
Epoch 7/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 3

In [ ]:
# ==============================
# Hybrid EfficientNet + Transformer
# ==============================

IMG_SIZE = (300, 300)
BATCH_SIZE = 32
SEED = 42
EPOCHS = 20

train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(AUTOTUNE)
val_ds   = val_ds.cache().prefetch(AUTOTUNE)

base_model = tf.keras.applications.EfficientNetB3(
    weights='imagenet',
    include_top=False,
    input_shape=(300,300,3)
)
base_model.trainable = False

inputs = keras.Input(shape=(300,300,3))
x = layers.Rescaling(1./255)(inputs)
x = base_model(x, training=False)
x = layers.Reshape((-1, x.shape[-1]))(x)

# Transformer block
attention = layers.MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
x = layers.Add()([x, attention])
x = layers.LayerNormalization()(x)

x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = keras.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

val_loss, val_acc = model.evaluate(val_ds)
print("✅ Validation Accuracy:", val_acc)
print("✅ Validation Loss:", val_loss)

Found 3089 files belonging to 9 classes.
Using 2472 files for training.
Found 3089 files belonging to 9 classes.
Using 617 files for validation.
43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 135s 988ms/step - accuracy: 0.2191 - loss: 2.1774 - val_accuracy: 0.2480 - val_loss: 1.9289
Epoch 2/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 13s 164ms/step - accuracy: 0.2495 - loss: 2.0142 - val_accuracy: 0.2545 - val_loss: 1.9387
Epoch 3/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 12s 160ms/step - accuracy: 0.2453 - loss: 1.9738 - val_accuracy: 0.2853 - val_loss: 1.9097
Epoch 4/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 12s 156ms/step - accuracy: 0.2464 - loss: 1.9772 - val_accuracy: 0.2480 - val_loss: 1.8922
Epoch 5/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 12s 153ms/step - accuracy: 0.2706 - loss: 1.9618 - val_accuracy: 0.2480 - val_loss: 1.8710
Epoch 6/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 12s 154ms/step - accuracy: 0.2442 - loss: 1.9370 - val_accuracy: 0.3031 - val_loss: 1.8857
Epoch 7/20
78/78 ━━━━━━━━━━━━